In [2]:
import numpy as np
import pandas as pd

df=pd.read_csv('Main_Data.csv')

#Data Overview
print(df.info())
print(df.head())
print(df.describe())

#Removing columns that are not needed for analysis
columns_to_remove = ['Rowid', 'StratificationCategoryID1', 'StratificationID1', 'LocationID', 'ResponseID','DataSource', 'Data_Value_Footnote_Symbol',
                    'Data_Value_Footnote', 'DataValueTypeID',
                    'Data_Value_Alt', 'CategoryID', 'IndicatorID']

df.drop(columns=columns_to_remove, inplace=True)
print("\nColumns after removal:")
print(df.columns)
print(df.info())
print(df.isnull().sum())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rowid                       2258 non-null   float64
 1   Year                        11238 non-null  int64  
 2   LocationAbbr                11238 non-null  object 
 3   LocationDesc                11238 non-null  object 
 4   DataSource                  11238 non-null  object 
 5   Category                    11238 non-null  object 
 6   Indicator                   11238 non-null  object 
 7   Response                    11124 non-null  object 
 8   Data_Value_Unit             11238 non-null  object 
 9   Data_Value_Type             11238 non-null  object 
 10  Data_Value                  10077 non-null  float64
 11  Data_Value_Alt              10077 non-null  float64
 12  Data_Value_Footnote_Symbol  1161 non-null   object 
 13  Data_Value_Footnote         116

In [3]:
import re

# fill missing Geolocation with random POINTs between mean and max of existing coordinates

geo = df['Geolocation'].dropna().astype(str)
coords = geo.str.extract(r'POINT\s*\(\s*([-\d\.]+)\s+([-\d\.]+)\s*\)')
coords = coords.dropna().astype(float)

if coords.empty:
    raise ValueError("No valid 'POINT (lon lat)' Geolocation values found to compute ranges.")

lon_mean, lat_mean = coords[0].mean(), coords[1].mean()
lon_max, lat_max = coords[0].max(), coords[1].max()

# ensure non-zero range
if lon_mean == lon_max:
    lon_max = lon_mean + 1e-6
if lat_mean == lat_max:
    lat_max = lat_mean + 1e-6

mask = df['Geolocation'].isnull()
n = mask.sum()
rand_lons = np.random.uniform(lon_mean, lon_max, size=n)
rand_lats = np.random.uniform(lat_mean, lat_max, size=n)
filled = ['POINT ({:.12f} {:.12f})'.format(lon, lat) for lon, lat in zip(rand_lons, rand_lats)]
df.loc[mask, 'Geolocation'] = filled

print(f'Filled {n} missing Geolocation values.')

#Status
print(df.info())
print(df.isnull().sum())


Filled 9335 missing Geolocation values.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       11238 non-null  int64  
 1   LocationAbbr               11238 non-null  object 
 2   LocationDesc               11238 non-null  object 
 3   Category                   11238 non-null  object 
 4   Indicator                  11238 non-null  object 
 5   Response                   11124 non-null  object 
 6   Data_Value_Unit            11238 non-null  object 
 7   Data_Value_Type            11238 non-null  object 
 8   Data_Value                 10077 non-null  float64
 9   Low_Confidence_Limit       10077 non-null  float64
 10  High_Confidence_Limit      10077 non-null  float64
 11  Number                     10077 non-null  float64
 12  WeightedNumber             10077 non-null  float64
 13  Strati

In [4]:
# Create a mapping of StratificationCategory2 to its possible Stratification2 values
category_mapping = {}
for cat in df['StratificationCategory2'].dropna().unique():
    values = df[df['StratificationCategory2'] == cat]['Stratification2'].dropna().unique()
    category_mapping[cat] = values

print(f"Created category_mapping with {len(category_mapping)} categories")
print(category_mapping)

Created category_mapping with 3 categories
{'Sex': array(['Female', 'Male'], dtype=object), 'Age Group': array(['18-44', '45-64', '65+', '51-65', '21-35', '25-44', '36-50',
       '18-24', '20-44'], dtype=object), 'Race/Ethnicity': array(['Hispanic', 'Black, non-Hispanic',
       'Other / Multirace, non-Hispanic', 'White, non-Hispanic'],
      dtype=object)}


In [5]:
# Fill missing StratificationCategory2 and Stratification2
mask_fill = df['StratificationCategory2'].isnull()
n_missing = mask_fill.sum()

# Get categories and their values
categories = list(category_mapping.keys())

# Randomly assign categories and corresponding values
np.random.seed(42)  # for reproducibility
assigned_categories = np.random.choice(categories, size=n_missing)
assigned_values = [
    np.random.choice(category_mapping[cat])
    for cat in assigned_categories
]

# Fill the missing values
df.loc[mask_fill, 'StratificationCategory2'] = assigned_categories
df.loc[mask_fill, 'Stratification2'] = assigned_values

print(f"Filled {n_missing} missing values in StratificationCategory2 and Stratification2")
print(df.info())
print(df.isnull().sum())

Filled 3613 missing values in StratificationCategory2 and Stratification2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       11238 non-null  int64  
 1   LocationAbbr               11238 non-null  object 
 2   LocationDesc               11238 non-null  object 
 3   Category                   11238 non-null  object 
 4   Indicator                  11238 non-null  object 
 5   Response                   11124 non-null  object 
 6   Data_Value_Unit            11238 non-null  object 
 7   Data_Value_Type            11238 non-null  object 
 8   Data_Value                 10077 non-null  float64
 9   Low_Confidence_Limit       10077 non-null  float64
 10  High_Confidence_Limit      10077 non-null  float64
 11  Number                     10077 non-null  float64
 12  WeightedNumber             1

In [6]:
#Fill missing StratificationCategoryID2 and StratificationID2
mask_fill_ids = df['StratificationCategoryID2'].isnull()
n_missing_ids = mask_fill_ids.sum()

#Create mapping for category IDs and value IDs based on existing data
category_to_id = {
    'Sex': 'SEX',
    'Race/Ethnicity': 'RACE',
    'Age Group': 'AGEGRP'
}

value_to_id = {
    'Female': 'F',
    'Male': 'M',
    'Black, non-Hispanic': 'BLK',
    'Hispanic': 'HIS',
    'White, non-Hispanic': 'WHT',
    'Other / Multirace, non-Hispanic': 'MRC',
    '65+': 'AGE03',
    '45-64': 'AGE02',
    '25-44': 'AGE05',
    '18-44': 'AGE01',
    '20-44': 'AGE06',
    '18-24': 'AGE04',
    '51-65': 'AGE09',
    '21-35': 'AGE08',
    '36-50': 'AGE02'
}

# Map the existing StratificationCategory2 and Stratification2 values to their IDs
rows_to_fill = df[mask_fill_ids].index
assigned_category_ids = [category_to_id.get(df.loc[idx, 'StratificationCategory2'], 'UNK') for idx in rows_to_fill]
assigned_value_ids = [value_to_id.get(df.loc[idx, 'Stratification2'], 'UNK') for idx in rows_to_fill]

#Fill the missing values
df.loc[mask_fill_ids, 'StratificationCategoryID2'] = assigned_category_ids
df.loc[mask_fill_ids, 'StratificationID2'] = assigned_value_ids

print(f"Filled {n_missing_ids} missing values in StratificationCategoryID2 and StratificationID2")
print(df.info())
print(df.isnull().sum())

Filled 3613 missing values in StratificationCategoryID2 and StratificationID2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       11238 non-null  int64  
 1   LocationAbbr               11238 non-null  object 
 2   LocationDesc               11238 non-null  object 
 3   Category                   11238 non-null  object 
 4   Indicator                  11238 non-null  object 
 5   Response                   11124 non-null  object 
 6   Data_Value_Unit            11238 non-null  object 
 7   Data_Value_Type            11238 non-null  object 
 8   Data_Value                 10077 non-null  float64
 9   Low_Confidence_Limit       10077 non-null  float64
 10  High_Confidence_Limit      10077 non-null  float64
 11  Number                     10077 non-null  float64
 12  WeightedNumber          

In [7]:
# Drop the last two columns (StratificationCategoryID2 and StratificationID2)
df.drop(df.columns[-2:], axis=1, inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     11238 non-null  int64  
 1   LocationAbbr             11238 non-null  object 
 2   LocationDesc             11238 non-null  object 
 3   Category                 11238 non-null  object 
 4   Indicator                11238 non-null  object 
 5   Response                 11124 non-null  object 
 6   Data_Value_Unit          11238 non-null  object 
 7   Data_Value_Type          11238 non-null  object 
 8   Data_Value               10077 non-null  float64
 9   Low_Confidence_Limit     10077 non-null  float64
 10  High_Confidence_Limit    10077 non-null  float64
 11  Number                   10077 non-null  float64
 12  WeightedNumber           10077 non-null  float64
 13  StratificationCategory1  11238 non-null  object 
 14  Stratification1       

In [8]:
mode_vals = df['Response'].mode(dropna=True)


n_missing = df['Response'].isnull().sum()
df['Response'].fillna(mode_vals[0], inplace=True)

print(f"Filled {n_missing} missing 'Response' values with mode: {mode_vals[0]}")
print(df.info())
print(df.isnull().sum())


Filled 114 missing 'Response' values with mode: Yes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     11238 non-null  int64  
 1   LocationAbbr             11238 non-null  object 
 2   LocationDesc             11238 non-null  object 
 3   Category                 11238 non-null  object 
 4   Indicator                11238 non-null  object 
 5   Response                 11238 non-null  object 
 6   Data_Value_Unit          11238 non-null  object 
 7   Data_Value_Type          11238 non-null  object 
 8   Data_Value               10077 non-null  float64
 9   Low_Confidence_Limit     10077 non-null  float64
 10  High_Confidence_Limit    10077 non-null  float64
 11  Number                   10077 non-null  float64
 12  WeightedNumber           10077 non-null  float64
 13  StratificationCategory1 

C:\Users\praya\AppData\Local\Temp\ipykernel_24068\4251179389.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Response'].fillna(mode_vals[0], inplace=True)


In [9]:
# Fill missing Data_Value with random values between min and max
n_missing_data_value = df['Data_Value'].isnull().sum()
# Get min and max from non-null values
min_val = df['Data_Value'].min()
max_val = df['Data_Value'].max()
    
print(f"Data_Value range: {min_val} to {max_val}")
print(f"Filling {n_missing_data_value} missing Data_Value entries")
    
# Fill with random values in the range
mask = df['Data_Value'].isnull()
rand_values = np.random.uniform(min_val, max_val, size=n_missing_data_value)
df.loc[mask, 'Data_Value'] = rand_values
   

print(df.info())
print(df.isnull().sum())

Data_Value range: 0.7 to 99.9
Filling 1161 missing Data_Value entries
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     11238 non-null  int64  
 1   LocationAbbr             11238 non-null  object 
 2   LocationDesc             11238 non-null  object 
 3   Category                 11238 non-null  object 
 4   Indicator                11238 non-null  object 
 5   Response                 11238 non-null  object 
 6   Data_Value_Unit          11238 non-null  object 
 7   Data_Value_Type          11238 non-null  object 
 8   Data_Value               11238 non-null  float64
 9   Low_Confidence_Limit     10077 non-null  float64
 10  High_Confidence_Limit    10077 non-null  float64
 11  Number                   10077 non-null  float64
 12  WeightedNumber           10077 non-null  float64
 13  Strati

In [10]:
# Fill missing Low_Confidence_Limit with the median of the column
n_missing_lcl = df['Low_Confidence_Limit'].isnull().sum()
median_lcl = df['Low_Confidence_Limit'].median()
df['Low_Confidence_Limit'].fillna(median_lcl, inplace=True)
print(f"Filled {n_missing_lcl} missing Low_Confidence_Limit values with median: {median_lcl}")
print(df.info())
print(df.isnull().sum())

Filled 1161 missing Low_Confidence_Limit values with median: 30.9
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     11238 non-null  int64  
 1   LocationAbbr             11238 non-null  object 
 2   LocationDesc             11238 non-null  object 
 3   Category                 11238 non-null  object 
 4   Indicator                11238 non-null  object 
 5   Response                 11238 non-null  object 
 6   Data_Value_Unit          11238 non-null  object 
 7   Data_Value_Type          11238 non-null  object 
 8   Data_Value               11238 non-null  float64
 9   Low_Confidence_Limit     11238 non-null  float64
 10  High_Confidence_Limit    10077 non-null  float64
 11  Number                   10077 non-null  float64
 12  WeightedNumber           10077 non-null  float64
 13  Stratifica

C:\Users\praya\AppData\Local\Temp\ipykernel_24068\1911981695.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Low_Confidence_Limit'].fillna(median_lcl, inplace=True)


In [11]:
# Fill missing High_Confidence_Limit with the median of the column
n_missing_hcl = df['High_Confidence_Limit'].isnull().sum()
median_hcl = df['High_Confidence_Limit'].median()
df['High_Confidence_Limit'].fillna(median_hcl, inplace=True)
print(f"Filled {n_missing_hcl} missing High_Confidence_Limit values with median: {median_hcl}")
print(df.info())
print(df.isnull().sum())

Filled 1161 missing High_Confidence_Limit values with median: 43.6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     11238 non-null  int64  
 1   LocationAbbr             11238 non-null  object 
 2   LocationDesc             11238 non-null  object 
 3   Category                 11238 non-null  object 
 4   Indicator                11238 non-null  object 
 5   Response                 11238 non-null  object 
 6   Data_Value_Unit          11238 non-null  object 
 7   Data_Value_Type          11238 non-null  object 
 8   Data_Value               11238 non-null  float64
 9   Low_Confidence_Limit     11238 non-null  float64
 10  High_Confidence_Limit    11238 non-null  float64
 11  Number                   10077 non-null  float64
 12  WeightedNumber           10077 non-null  float64
 13  Stratific

C:\Users\praya\AppData\Local\Temp\ipykernel_24068\2672733192.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['High_Confidence_Limit'].fillna(median_hcl, inplace=True)


In [12]:
# Convert Number and WeightedNumber to float
for col in ['Number', 'WeightedNumber']:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '', regex=False), errors='coerce').astype(float)
    print(f"{col} dtype: {df[col].dtype}")

print(df[['Number', 'WeightedNumber']].info())

Number dtype: float64
WeightedNumber dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Number          10077 non-null  float64
 1   WeightedNumber  10077 non-null  float64
dtypes: float64(2)
memory usage: 175.7 KB
None


In [13]:
# Fill missing Number and WeightedNumber with random values
for col in ['Number', 'WeightedNumber']:
    mean_val = df[col].mean()
    n_missing = df[col].isnull().sum()
    if n_missing > 0:
        mask = df[col].isnull()
        rand_values = np.random.uniform(df[col].min(), mean_val, size=n_missing)
        df.loc[mask, col] = rand_values

print(df.isnull().sum())

Year                       0
LocationAbbr               0
LocationDesc               0
Category                   0
Indicator                  0
Response                   0
Data_Value_Unit            0
Data_Value_Type            0
Data_Value                 0
Low_Confidence_Limit       0
High_Confidence_Limit      0
Number                     0
WeightedNumber             0
StratificationCategory1    0
Stratification1            0
StratificationCategory2    0
Stratification2            0
Geolocation                0
dtype: int64


In [14]:
#Drop Low_Confidence_Limit and High_Confidence_Limit columns
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     11238 non-null  int64  
 1   LocationAbbr             11238 non-null  object 
 2   LocationDesc             11238 non-null  object 
 3   Category                 11238 non-null  object 
 4   Indicator                11238 non-null  object 
 5   Response                 11238 non-null  object 
 6   Data_Value_Unit          11238 non-null  object 
 7   Data_Value_Type          11238 non-null  object 
 8   Data_Value               11238 non-null  float64
 9   Low_Confidence_Limit     11238 non-null  float64
 10  High_Confidence_Limit    11238 non-null  float64
 11  Number                   11238 non-null  float64
 12  WeightedNumber           11238 non-null  float64
 13  StratificationCategory1  11238 non-null  object 
 14  Stratification1       

In [15]:
# extract lon/lat from 'POINT (lon lat)' into separate float columns
pattern = r'POINT\s*\(\s*([-\d\.]+)[,\s]+([-\d\.]+)\s*\)'
coords = df['Geolocation'].astype(str).str.extract(pattern)
coords.columns = ['Longitude', 'Latitude']
df['Longitude'] = pd.to_numeric(coords['Longitude'], errors='coerce')
df['Latitude'] = pd.to_numeric(coords['Latitude'], errors='coerce')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     11238 non-null  int64  
 1   LocationAbbr             11238 non-null  object 
 2   LocationDesc             11238 non-null  object 
 3   Category                 11238 non-null  object 
 4   Indicator                11238 non-null  object 
 5   Response                 11238 non-null  object 
 6   Data_Value_Unit          11238 non-null  object 
 7   Data_Value_Type          11238 non-null  object 
 8   Data_Value               11238 non-null  float64
 9   Low_Confidence_Limit     11238 non-null  float64
 10  High_Confidence_Limit    11238 non-null  float64
 11  Number                   11238 non-null  float64
 12  WeightedNumber           11238 non-null  float64
 13  StratificationCategory1  11238 non-null  object 
 14  Stratification1       

In [16]:
df.drop(columns=['High_Confidence_Limit', 'Low_Confidence_Limit'], inplace=True)

In [17]:

print(df.describe())

               Year    Data_Value         Number  WeightedNumber  \
count  11238.000000  11238.000000   11238.000000    1.123800e+04   
mean    2018.559263     43.442380    1108.266195    6.303382e+05   
std        2.132482     28.225218    5790.526413    3.381437e+06   
min     2016.000000      0.700000       8.000000    4.390000e+02   
25%     2017.000000     19.808190     122.000000    4.327775e+04   
50%     2018.000000     38.200000     344.000000    1.486760e+05   
75%     2020.000000     67.500000     860.946351    4.435572e+05   
max     2022.000000     99.900000  236045.000000    1.584285e+08   

          Longitude      Latitude  
count  11238.000000  11238.000000  
mean     -83.250749     50.113880  
std       11.098821      8.492347  
min     -157.857749     21.304850  
25%      -88.661065     43.429034  
50%      -82.018530     49.845531  
75%      -75.577741     57.288751  
max      -68.985031     64.845080  


In [18]:
# Export cleaned dataframe to CSV
out_path = 'DHDS_cleaned.csv'
df.to_csv(out_path, index=False)
print(f"Saved cleaned data to {out_path}")

Saved cleaned data to DHDS_cleaned.csv
